In [ ]:
import os 

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten, MaxPooling2D, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import joblib

In [ ]:
data_path = r'D:\dbda\project\FER Model\CK+48'

In [ ]:
baseModel = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)


for layer in baseModel.layers:
  layer.trainable = False

headModel = baseModel.output
headModel = MaxPooling2D()(headModel)
headModel = Flatten()(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(7, activation='softmax')(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   rotation_range=15,
                                   horizontal_flip = True)


training_set = train_datagen.flow_from_directory(data_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                 shuffle=True)



In [ ]:
model.fit_generator(training_set,
                              epochs=20,
                             steps_per_epoch = len(training_set))

# Saving Model

In [ ]:
model_json = model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("emotion_model.h5")